<a href="https://colab.research.google.com/github/gonzalezjulvez/Projects/blob/main/Formacion/Clasificaci%C3%B3n_de_texto_b%C3%A1sica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación de texto básica

En este tutorial demuestra la clasificación de texto a partir de archivos de texto sin formato almacenados en el disco. Entrenaremos a un clasificador binario para realizar análisis de sentimientos en un conjunto de datos IMDB. Al final del notebooks, en el que entrenaremos a un clasificador multiclase para predecir la etiqueta de una pregunta de programación en Stack Overflow.

Este cuaderno entrena un modelo de análisis de sentimientos para clasificar las reseñas de películas como positivas o negativas , según el texto de la reseña. Este es un ejemplo de clasificación binaria, o de dos clases, un tipo de problema de aprendizaje automático importante y ampliamente aplicable.
Utilizará el conjunto de datos de reseñas de películas grandes que contiene el texto de 50.000 reseñas de películas de Internet Movie Database. Estos se dividen en 25,000 revisiones para capacitación y 25,000 revisiones para pruebas. Los conjuntos de entrenamiento y prueba están equilibrados , lo que significa que contienen un número igual de críticas positivas y negativas.




## Cargamos las librerias 

In [12]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from pathlib import Path

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


## Cargamos el Datasets 

In [3]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz',url,untar=True, cache_dir='.',cache_subdir='')
dataset_dir=os.path.join(os.path.dirname(dataset), 'aclImdb')

84131840/84125825 [==============================] - 12s 0us/step


In [5]:
os.listdir(dataset_dir)

['train', 'README', 'test', 'imdb.vocab', 'imdbEr.txt']

In [10]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['pos',
 'urls_pos.txt',
 'labeledBow.feat',
 'unsupBow.feat',
 'neg',
 'urls_unsup.txt',
 'unsup',
 'urls_neg.txt']

Los aclImdb/train/pos y aclImdb/train/neg contienen muchos archivos de texto, cada uno de los cuales es una reseña de una sola película.

In [11]:
sample_file = os.path.join(train_dir,'pos/1181_9.txt')
with open(sample_file) as f:
  print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


Para preparar un conjunto de datos para la clasificación binaria, necesitará dos carpetas en el disco, correspondientes a class_a y class_b . Estas serán las críticas positivas y negativas de películas, que se pueden encontrar en aclImdb/train/pos y aclImdb/train/neg . Como el conjunto de datos de IMDB contiene carpetas adicionales, las eliminará antes de usar esta utilidad.



In [22]:
remove_dir = os.path.join(train_dir,'unsup')
shutil.rmtree(remove_dir)

A continuación, utilizará la utilidad text_dataset_from_directory para crear untf.data.Dataset etiquetado. tf.data es una poderosa colección de herramientas para trabajar con datos. Al ejecutar un experimento de aprendizaje automático, se recomienda dividir su conjunto de datos en tres divisiones: entrenamiento , validación y prueba .

El conjunto de datos de IMDB ya se ha dividido en entrenamiento y prueba, pero carece de un conjunto de validación. Creemos un conjunto de validación usando una división 80:20 de los datos de entrenamiento usando el argumento validation_split continuación.

In [23]:
batch_size = 32
seed= 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory('/content/aclImdb/train',
                                                                  batch_size=batch_size,
                                                                  validation_split=0.2,
                                                                  subset='training',
                                                                  seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


Como puede ver arriba, hay 25.000 ejemplos en la carpeta de formación, de los cuales utilizará el 80% (o 20.000) para la formación. Como verá en un momento, puede entrenar un modelo pasando un conjunto de datos directamente a model.fit . Si es nuevo en tf.data , también puede iterar sobre el conjunto de datos e imprimir algunos ejemplos de la siguiente manera.



In [25]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print(f"Review {text_batch.numpy()[i]}")
    print(f"Label {label_batch.numpy()[i]}")

Review b"Having seen most of Ringo Lam's films, I can say that this is his best film to date, and the most unusual. It's a ancient china period piece cranked full of kick-ass martial arts, where the location of an underground lair full of traps and dungeons plays as big a part as any of the characters. The action is fantastic, the story is tense and entertaining, and the set design is truely memorable. Sadly, Burning Paradise has not been made available on DVD and vhs is next-to-impossible to get your mitts on, even if you near the second biggest china-town in North America (like I do). If you can find it, don't pass it up."
Label 1
Review b'Caution: May contain spoilers...<br /><br />I\'ve seen this movie 3 times & I\'ve liked it every time. Upon seeing it again, I\'m always reminded of how good it is. An HBO TV movie- very well done like most of their movies are- this would\'ve gotten Oscars for it\'s performances had it been released for general distribution instead of made for TV.<

Observe que las reseñas contienen texto sin formato (con puntuación y etiquetas HTML ocasionales como). Mostrará cómo manejarlos en la siguiente sección.

Las etiquetas son 0 o 1. Para ver cuáles corresponden a reseñas de películas positivas y negativas, puede verificar la propiedad class_names en el conjunto de datos.



In [26]:
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Label 0 corresponds to neg
Label 1 corresponds to pos


AttributeError: ignored